In [1]:
#!/usr/bin/env python
# coding: utf-8
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import gc
import decoupler as dc
import regex as re
import decimal as dec
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
os.chdir("/Volumes/RicoData2/ReHeat2/")

ModuleNotFoundError: No module named 'decoupler'

In [2]:
all_datasets = [f for f in os.listdir("./simplified")]
study_id = [f.replace('.h5ad', "") for f in all_datasets]
pbulk_files = ["./pbulk/" + f + "_pbulk.csv" for f in study_id]
coldata_files = ["./coldata/" + f + "_coldata.csv" for f in study_id]
metadata_files = ["./metadata/" + f + "_metadata.csv" for f in study_id]
all_datasets = ["./simplified/" + f for f in all_datasets]

In [3]:
banned_md_columns = ["cell_type", "cell_states", "Sample"]

In [4]:
for s_id,s_f,c_f,m_f,pb_f in zip(study_id, all_datasets, coldata_files, metadata_files, pbulk_files):
    print("processing " + s_id)
    print("reading " + s_f)
    sc_dat = sc.read_h5ad(filename = s_f)

    # Creating meta_data
    meta_columns = [c for c in sc_dat.obs.columns if c not in banned_md_columns]
    meta_data = sc_dat.obs[meta_columns].drop_duplicates()
    print("n samples")
    print(sc_dat.obs[["sample_id"]].drop_duplicates().shape)
    print("shape meta")
    print(meta_data.shape)
    meta_data.to_csv(m_f)

    # Cell-type numbers needed for MOFAcell in coldata
    cell_type_numbers = sc_dat.obs.groupby(["sample_id", "cell_type"])["cell_type"].count()
    cell_type_numbers = cell_type_numbers.to_frame().rename({"cell_type":"counts"}, axis=1).reset_index()

    # Pbulk
    # To avoid crashes of matrix type
    if isinstance(sc_dat.X, csc_matrix):
        sc_dat.layers['counts'] = sc_dat.X.tocsr(copy = True)
    else:
        sc_dat.layers['counts'] = sc_dat.X
        
    padata = dc.get_pseudobulk(sc_dat, sample_col='sample_id', groups_col='cell_type', layer='counts', min_prop=0, min_smpls=0)
    pb_dat = pd.DataFrame(padata.X)
    pb_dat.columns = padata.var.index.values
    pb_dat.index = padata.obs.index.values
    pb_dat.to_csv(pb_f)
    print("Pseudobulk dimensions")
    print(pb_dat.shape)

    #Coldata
    pb_coldata = padata.obs.copy()
    pb_coldata["colname"] = pb_coldata.index.values
    pb_coldata = pb_coldata.merge(cell_type_numbers, on = ["cell_type", "sample_id"], how = "left")
    
    pb_coldata.to_csv(c_f)
    print("Coldata dimensions")
    print(pb_coldata.shape)
    
    

processing Armute2023_LVAD
reading ./simplified/Armute2023_LVAD.h5ad
n samples
(40, 1)
shape meta
(40, 6)
Pseudobulk dimensions
(280, 31031)
Coldata dimensions
(280, 11)
processing Reichart2022_DCM
reading ./simplified/Reichart2022_DCM.h5ad
n samples
(79, 1)
shape meta
(79, 8)
Pseudobulk dimensions
(550, 30602)
Coldata dimensions
(550, 13)
processing Kuppe2022_MI
reading ./simplified/Kuppe2022_MI.h5ad
n samples
(27, 1)
shape meta
(27, 7)
Pseudobulk dimensions
(189, 28990)
Coldata dimensions
(189, 12)
processing Koenig2022_DCM
reading ./simplified/Koenig2022_DCM.h5ad
n samples
(38, 1)
shape meta
(38, 4)
Pseudobulk dimensions
(266, 31377)
Coldata dimensions
(266, 9)
processing Chaffin2022_DCM
reading ./simplified/Chaffin2022_DCM.h5ad
n samples
(42, 1)
shape meta
(42, 7)
Pseudobulk dimensions
(294, 33364)
Coldata dimensions
(294, 12)
processing Simonson2023_ICM
reading ./simplified/Simonson2023_ICM.h5ad
n samples
(15, 1)
shape meta
(15, 6)
Pseudobulk dimensions
(105, 31387)
Coldata dimens